# Global Configuration

This notebook example demonstrates how the global configuration object, the `MassConfiguration`, can be used to configure default behavior for various **MASSpy** methods as well as the default behavior for **COBRApy**.

In [ ]:
import cobra

import mass
from mass.test import create_test_model

cobra_config = cobra.Configuration()

Note that changing global configuration values is mostly useful at the beginning of a work session.

## The MassConfiguration Object

Just like the `cobra.Configuration` object, the `MassConfiguration` object is a [singleton](https://en.wikipedia.org/wiki/Singleton_pattern), meaning that only one instance can exist and therefore is respected everywhere in **MASSpy**. 

The `MassConfiguration` can retrieved via the following:

In [ ]:
mass_config = mass.MassConfiguration()

The `MassConfiguration` is synchronized with the `cobra.Configuration` singleton object such that a change in one configuration object affects the other.

In [ ]:
print("cobra configuration before: {0!r}".format(cobra_config.bounds))
mass_config.bounds = (-444, 444)
print("cobra configuration after: {0!r}".format(cobra_config.bounds))

This means that changes only need to be made to the `MassConfiguration` object for workflows involving both the **COBRApy** and **MASSpy** package. The shared values can be viewed using the `shared_state` attribute

In [ ]:
mass_config.shared_state

## Attributes for Model Construction

The following attributes of the `MassConfiguration` alter default behavior for constructing models as well as importing/exporting models via [SBML](http://sbml.org/Main_Page).

In [ ]:
from mass import MassMetabolite, MassReaction

### For Irreversible Reactions

When an irreversible reaction is created, the equilibrium constant and reverse rate constant are automatically set based on the `irreversible_Keq` and `irreversible_kr` attributes, respectively.

In [ ]:
print("Irreversible Keq: {0}".format(mass_config.irreversible_Keq))
print("Irreversible kr: {0}".format(mass_config.irreversible_kr))
R1 = MassReaction("R1", reversible=False)
R1.parameters

Changing the `irreversible_Keq` and `irreversible_kr` attributes affects newly created `MassReaction` objects.

In [ ]:
mass_config.irreversible_Keq = 10e6
mass_config.irreversible_kr = 1e-6
print("Irreversible Keq: {0}".format(mass_config.irreversible_Keq))
print("Irreversible kr: {0}\n".format(mass_config.irreversible_kr))

# Create new reaction
R2 = MassReaction("R2", reversible=False)
print(R2.parameters)

Existing reactions are not affected.

In [ ]:
print(R1.parameters)

### For Rate Expressions

Automatic generation of rate expressions can be affected using the `exclude_metabolites_from_rates` and `exclude_compartment_volumes_in_rates` attributes.

In [ ]:
model = create_test_model("textbook")

The `exclude_metabolites_from_rates` determines which metabolites to exclude from rate expressions using a dictionary containing the metabolite attribute to use for filtering, and the values to be excluded.

In [ ]:
mass_config.exclude_metabolites_from_rates

The default setting is to use the `elements` attribute of `MassMetabolite` objects and excludes any metabolite that returns the elements for hydrogen and for water.

In [ ]:
ENO = model.reactions.get_by_id("ENO")
print(ENO.rate)

The `exclude_metabolites_from_rates` can be changed by passing a `dict` containing a metabolite attribute and a list of values to be excluded. For example, to exclude "2pg_c" using its name as a criteria:

In [ ]:
mass_config.exclude_metabolites_from_rates = {"name": ["D-Glycerate 2-phosphate"]}
ENO = model.reactions.get_by_id("ENO")
print(ENO.rate)

Or to exclude hydrogen and water using their identifiers:

In [ ]:
mass_config.exclude_metabolites_from_rates = {"id": ["h_c", "h2o_c"]}
ENO = model.reactions.get_by_id("ENO")
print(ENO.rate)

Boundary reactions are not affected by the `exclude_metabolites_from_rates` attribute:

In [ ]:
for rid in ["SK_h_c", "SK_h2o_c"]:
    reaction = model.reactions.get_by_id(rid)
    print(reaction.rate)

The `exclude_compartment_volumes_in_rates` attribute determines whether compartment volumes should be factored into rate expressions. By default, compartment volumes are not included in automatically generated rate expressions 

In [ ]:
PGI = model.reactions.get_by_id("PGI")
print(PGI.rate)

When `exclude_compartment_volumes_in_rates` is set as `False`, compartments are included in rate expressions as "volume_CID", where "CID" refers to the compartment identifier. 

In [ ]:
mass_config.exclude_compartment_volumes_in_rates = False

PGI = model.reactions.get_by_id("PGI")
model.custom_parameters["volume_c"] = 1

print(PGI.rate)

The compartment volume is currently treated as a custom parameter. This behavior is subject to change in future updates following the release of COBRApy compartment objects. 

### For Compartments and SBML

The `boundary_compartment` attribute defines the compartment for any external boundary species. 

In [ ]:
# Create a boundary reaction
x1_c = MassMetabolite("x1_c", compartment="c")
R3 = MassReaction("R1")
R3.add_metabolites({x1_c: -1})

print(mass_config.boundary_compartment)
R3.boundary_metabolite

The `boundary_compartment` can be changed using a `dict`, where the key is the compartment identifier and the value is the compartment name.

In [ ]:
mass_config.boundary_compartment = {"xt": "external"}
R3.boundary_metabolite

Because the `mass.Simulation` object uses the `libroadrunner`, an integrator for SBML models, a model cannot be simulated without defining a compartment. The `default_compartment` attribute is used as the compartment for the model when no compartments have been defined.

In [ ]:
mass_config.default_compartment

As with the `boundary_compartment`, the `default_compartment` can be changed using a `dict`.

In [ ]:
mass_config.default_compartment = {"def": "default_compartment"}
mass_config.default_compartment

SBML also allows for a model creator to be defined when exporting models

In [ ]:
mass_config.model_creator

The `model_creator` attribute of the `MassConfiguration` allows the model creator to be set using a `dict`, where valid keys are "familyName", "givenName", "organization", and "email".

In [ ]:
mass_config.model_creator = {
    "familyName": "Smith",
    "givenName": "John",
    "organization": "Systems Biology Research Group @UCSD"}
mass_config.model_creator

## Attributes for Simulation and Analysis

The following attributes of the `MassConfiguration` alter default behavior for various simulation and analytical methods in **MASSPy**.

In [ ]:
from mass import Simulation

# Reset configurations before loading model
mass_config.boundary_compartment = {"b": "boundary"}
mass_config.exclude_compartment_volumes_in_rates = True

model = create_test_model("Glycolysis")
sim = Simulation(model, verbose=True)

### Steady State threshold

The `MassConfiguration.steady_state_threshold` attribute determines whether a model has reached a steady state. In general, compared values must be less than the `steady_state_threshold` attribute to be considered at steady state.

* With simulations. the absolute difference between the last two points must be less than the steady state threshold.

* With steady state solvers, the sum of squares of the steady state solution must be less than the steady state threshold.

In [ ]:
mass_config.steady_state_threshold = 1e-20
conc_sol, flux_sol = sim.find_steady_state(model, strategy="simulate")
bool(conc_sol)  # Empty solution objects return False

Changing the threshold affects whether solution values are considered to be at steady state:

In [ ]:
mass_config.steady_state_threshold = 1e-6
conc_sol, flux_sol = sim.find_steady_state(model, strategy="simulate")
bool(conc_sol)  # Filled solution objects return False

### Decimal Precision

The `MassConfiguration.decimal_precision` attribute is a special configuration attribute used in several `mass` methods. The value of the attribute determines how many digits after the decimal to preserve in rounding.

For many methods, the decimal precision will not be applied unless a `decimal_precision` kwarg is set as `True`. 

In [ ]:
# Set decimal precision
mass_config.decimal_precision = 8

# Will not apply decimal precision to steady state solutions
conc_sol, flux_sol = sim.find_steady_state(model, strategy="simulate",
                                           decimal_precision=False)
print(conc_sol["glc__D_c"])

# Will apply decimal precision to steady state solutions
conc_sol, flux_sol = sim.find_steady_state(model, strategy="simulate",
                                           decimal_precision=True)
print(conc_sol["glc__D_c"])

If `MassConfiguration.decimal_precision` is `None`, no rounding will occur.

In [ ]:
mass_config.decimal_precision = None

# Will apply decimal precision to steady state solutions
conc_sol, flux_sol = sim.find_steady_state(model, strategy="simulate",
                                           decimal_precision=True)
print(conc_sol["glc__D_c"])

## Shared Attributes

The following attributes are those shared with the `cobra.Configuration` object.

### Bounds

When a reaction is created, its default bounds are determined by the  `lower_bound` and `upper_bound` values of the `MassConfiguration`:

In [ ]:
mass_config.lower_bound = -1000
mass_config.upper_bound = 1000
R4 = MassReaction("R4")
print("R4 bounds: {0}".format(R4.bounds))

Changing the bounds will affect newly created reactions, but not existing ones:

In [ ]:
mass_config.bounds = (-444, 444)
R5 = MassReaction("R5")
print("R5 bounds: {0}".format(R5.bounds))
print("R4 bounds: {0}".format(R4.bounds))

### Solver

The default solver and solver tolerance utilized by newly instantiated models and `ConcSolver` objects is determined by the `solver` and `tolerance` attributes defined in the `MassConfiguration`.

In [ ]:
model = create_test_model("textbook")
print("Solver {0!r}".format(model.solver))
print("Tolerance {0}".format(model.tolerance))

The default solver can be changed depending on the solvers installed in the environment containing the **MASSpy** package. GLPK is assumed to always be present in the environment. 

The solver tolerance is similarly set using the `tolerance` attribute.

In [ ]:
# Change solver and solver tolerance
mass_config.solver = "glpk"
mass_config.tolerance = 1e-4

# Instantiate a new model to observe changes
model = create_test_model("textbook")
print("Solver {0!r}".format(model.solver))
print("Tolerance {0}".format(model.tolerance))

### Number of Processes

The `MassConfiguration.processes` controls the number of processes used when multiprocessing is possible. The default number corresponds to the number of available cores (hyperthreads).

In [ ]:
mass_config.processes